In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tqdm
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail
import requests
from nba_api.stats.static.players import find_players_by_full_name
from adjustText import adjust_text
from io import BytesIO
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import time
from PIL import Image
from collections import defaultdict

In [2]:
from nba_api.stats.endpoints.playbyplayv2 import PlayByPlayV2

from nba_api.stats.endpoints.playbyplay import PlayByPlay
from nba_api.stats.endpoints.leaguegamefinder import LeagueGameFinder
from nba_api.stats.static.players import find_players_by_full_name
from nba_api.stats.static.teams import find_teams_by_full_name
from nba_api.stats.endpoints.playergamelogs import PlayerGameLogs
from nba_api.stats.endpoints.boxscoretraditionalv2 import BoxScoreTraditionalV2
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail

from nba_api.stats.endpoints.leaguedashteamshotlocations import LeagueDashTeamShotLocations
from nba_api.stats.endpoints.leaguedashplayerptshot import LeagueDashPlayerPtShot

from nba_api.stats.endpoints.synergyplaytypes import SynergyPlayTypes
from nba_api.stats.endpoints.leaguedashptstats import LeagueDashPtStats
from nba_api.stats.endpoints.leaguedashplayerstats import LeagueDashPlayerStats

from nba_api.stats.endpoints.leaguehustlestatsplayer import LeagueHustleStatsPlayer

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests
import tqdm
from time import sleep
import time
from PIL import Image
from io import BytesIO
from adjustText import adjust_text
from scipy.interpolate import interp1d

from nba_api.stats.static.players import find_players_by_full_name
from nba_api.stats.static.teams import find_teams_by_full_name, get_teams, find_team_name_by_id
from nba_api.stats.endpoints.playbyplayv2 import PlayByPlayV2
from nba_api.stats.endpoints.leaguegamefinder import LeagueGameFinder
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail

from nba_api.stats.endpoints.synergyplaytypes import SynergyPlayTypes
from nba_api.stats.endpoints.leaguedashptstats import LeagueDashPtStats


In [4]:
giannis_id = find_players_by_full_name("Giannis Antetokounmpo")[0]['id']

In [9]:
shot_df = ShotChartDetail(team_id=0, player_id=giannis_id, season_type_all_star='Playoffs', context_measure_simple='FGA', season_nullable='2020-21').get_data_frames()[0]

In [12]:
shot_df.columns

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM'],
      dtype='object')

In [13]:
shot_df.c

0      In The Paint (Non-RA)
1      In The Paint (Non-RA)
2            Restricted Area
3            Restricted Area
4          Above the Break 3
               ...          
237    In The Paint (Non-RA)
238          Restricted Area
239          Restricted Area
240                Mid-Range
241    In The Paint (Non-RA)
Name: SHOT_ZONE_BASIC, Length: 242, dtype: object

In [18]:
shot_df.SHOT_ZONE_RANGE

0             8-16 ft.
1      Less Than 8 ft.
2      Less Than 8 ft.
3      Less Than 8 ft.
4              24+ ft.
            ...       
237           8-16 ft.
238    Less Than 8 ft.
239    Less Than 8 ft.
240           8-16 ft.
241    Less Than 8 ft.
Name: SHOT_ZONE_RANGE, Length: 242, dtype: object

In [19]:
shot_df.groupby(['GAME_ID', 'SHOT_ZONE_RANGE']).SHOT_ATTEMPTED_FLAG.sum()

GAME_ID     SHOT_ZONE_RANGE
0042000121  24+ ft.             3
            8-16 ft.            6
            Less Than 8 ft.    18
0042000122  24+ ft.             7
            8-16 ft.            1
            Less Than 8 ft.    15
0042000123  24+ ft.             2
            8-16 ft.            2
            Less Than 8 ft.     8
0042000124  16-24 ft.           2
            24+ ft.             4
            8-16 ft.            2
            Less Than 8 ft.    10
0042000211  16-24 ft.           2
            24+ ft.             5
            8-16 ft.            4
            Less Than 8 ft.    13
0042000212  16-24 ft.           1
            24+ ft.             3
            8-16 ft.            6
            Less Than 8 ft.     5
0042000213  16-24 ft.           2
            24+ ft.             8
            8-16 ft.            7
            Less Than 8 ft.    14
0042000214  24+ ft.             5
            8-16 ft.            4
            Less Than 8 ft.    17
0042000215  24+ ft. 

In [20]:
32 / (32+11+10+17)

0.45714285714285713

In [21]:
57 / (57+33)

0.6333333333333333

In [22]:
shot_df.ACTION_TYPE

0                  Driving Layup Shot
1                  Reverse Layup Shot
2                  Running Layup Shot
3                   Driving Dunk Shot
4                           Jump Shot
                    ...              
237                  Pullup Jump shot
238                Driving Layup Shot
239    Driving Finger Roll Layup Shot
240              Turnaround Jump Shot
241              Turnaround Hook Shot
Name: ACTION_TYPE, Length: 242, dtype: object

In [38]:
shot_types = shot_df.groupby(['GAME_ID', 'ACTION_TYPE']).SHOT_ATTEMPTED_FLAG.sum().to_frame().reset_index().pivot(columns='GAME_ID', index='ACTION_TYPE', values='SHOT_ATTEMPTED_FLAG').fillna(0.0)

In [34]:
nets_games = ['0042000211',
       '0042000212', '0042000213', '0042000214', '0042000215', '0042000216',
       '0042000217']

In [40]:
shot_types.loc[:, nets_games]

GAME_ID,0042000211,0042000212,0042000213,0042000214,0042000215,0042000216,0042000217
ACTION_TYPE,,,,,,,
Alley Oop Dunk Shot,1.0,0.0,0.0,1.0,0.0,0.0,2.0
Alley Oop Layup shot,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Cutting Dunk Shot,2.0,0.0,2.0,2.0,1.0,1.0,1.0
Cutting Layup Shot,0.0,0.0,0.0,1.0,1.0,0.0,0.0
Driving Dunk Shot,0.0,0.0,1.0,0.0,1.0,0.0,0.0
Driving Finger Roll Layup Shot,1.0,2.0,2.0,1.0,0.0,2.0,1.0
Driving Floating Bank Jump Shot,1.0,0.0,0.0,2.0,0.0,0.0,0.0
Driving Floating Jump Shot,2.0,0.0,0.0,1.0,1.0,0.0,0.0
Driving Layup Shot,1.0,2.0,5.0,0.0,5.0,5.0,5.0
